<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-exploration" data-toc-modified-id="Data-exploration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data exploration</a></span><ul class="toc-item"><li><span><a href="#Loading-the-data-and-formatting-the-variables-correctly" data-toc-modified-id="Loading-the-data-and-formatting-the-variables-correctly-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading the data and formatting the variables correctly</a></span></li><li><span><a href="#Strategy" data-toc-modified-id="Strategy-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Strategy</a></span></li><li><span><a href="#Base-Model" data-toc-modified-id="Base-Model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Base Model</a></span></li><li><span><a href="#New-heading" data-toc-modified-id="New-heading-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>New heading</a></span></li><li><span><a href="#Exploring-the-variables" data-toc-modified-id="Exploring-the-variables-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Exploring the variables</a></span><ul class="toc-item"><li><span><a href="#Numeric-variables" data-toc-modified-id="Numeric-variables-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Numeric variables</a></span></li><li><span><a href="#Categorical-variables" data-toc-modified-id="Categorical-variables-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Categorical variables</a></span></li></ul></li><li><span><a href="#Examing-the-missing-values-and-dropping-columns" data-toc-modified-id="Examing-the-missing-values-and-dropping-columns-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Examing the missing values and dropping columns</a></span></li><li><span><a href="#Handling-missing-values" data-toc-modified-id="Handling-missing-values-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Handling missing values</a></span></li><li><span><a href="#Exploring-the-correlation-between-variables" data-toc-modified-id="Exploring-the-correlation-between-variables-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Exploring the correlation between variables</a></span></li><li><span><a href="#Exploration-of-correlation-with-target-variable" data-toc-modified-id="Exploration-of-correlation-with-target-variable-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Exploration of correlation with target variable</a></span><ul class="toc-item"><li><span><a href="#We-look-at-the-categorical-variables" data-toc-modified-id="We-look-at-the-categorical-variables-1.9.1"><span class="toc-item-num">1.9.1&nbsp;&nbsp;</span>We look at the categorical variables</a></span></li></ul></li><li><span><a href="#Correlation-with-target-for-numerical-features" data-toc-modified-id="Correlation-with-target-for-numerical-features-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Correlation with target for numerical features</a></span></li></ul></li><li><span><a href="#Regression" data-toc-modified-id="Regression-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Regression</a></span><ul class="toc-item"><li><span><a href="#Splitting-in-train-and-testset" data-toc-modified-id="Splitting-in-train-and-testset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Splitting in train and testset</a></span></li><li><span><a href="#Different-regression-model" data-toc-modified-id="Different-regression-model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Different regression model</a></span><ul class="toc-item"><li><span><a href="#We-can-consider-this-problem-a-regression-problem-BUT-also-a-classificatino-problem-(-as-the-mood-is-integer-and-ordinal)" data-toc-modified-id="We-can-consider-this-problem-a-regression-problem-BUT-also-a-classificatino-problem-(-as-the-mood-is-integer-and-ordinal)-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>We can consider this problem a regression problem BUT also a classificatino problem ( as the mood is integer and ordinal)</a></span></li></ul></li><li><span><a href="#Performance-on-the-testset" data-toc-modified-id="Performance-on-the-testset-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Performance on the testset</a></span></li><li><span><a href="#Different-random-forest" data-toc-modified-id="Different-random-forest-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Different random forest</a></span></li></ul></li></ul></div>

# Data exploration

## Loading the data and formatting the variables correctly

In [ ]:
#libraries for exploration
import pandas as pd
#pandas settings
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_info_columns", 150)
from datetime import datetime

import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, Imputer, Normalizer
from scipy import stats

# plotting missing values
import missingno as msno

#plotting inline matplotlib
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline

# machine learning
from sklearn.linear_model import LinearRegression, ElasticNet

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import operator

#Visualisation of models
from yellowbrick.features import FeatureImportances
from yellowbrick.regressor import PredictionError, ResidualsPlot
from yellowbrick.regressor.alphas import AlphaSelection

In [ ]:
import sys
sys.path.append('../ML4QS/PythonCode')

In [ ]:
from util.VisualizeDataset import VisualizeDataset #cant get this to work

We load the data as a pandas dataframe and exampine the different features.

In [ ]:
mood = pd.read_csv("../Data/dataset_mood_smartphone.csv")

This is a temporal dataset without many features at first sight, however to use it in a predictive model we want to transform it. We can see that for every patient at different times different variables are logged, such as call activity, arousal, etc.

In [ ]:
mood.info()

We convert the time into actual pandas datetime format and also split date and time in different columns to help us further aggregate per time or per day.

In [ ]:
#Formatting of date time fields
def format_date(df):
    #parse string as datetime
    df["time"] = df["time"].apply(lambda x: \
    datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f"))  
    #split date and time so that we can use both lateer
    new_dates, new_times = zip(*[(d.date(), d.time()) for d in df['time']])
    df = df.assign(new_date=new_dates, new_time=new_times)
    df.drop("time", axis=1, inplace=True)
    return df

In [ ]:
mood = format_date(mood)

In [ ]:
mood.head(5)

We check the size of the data

In [ ]:
mood.shape

We have data for 27 unique patients

In [ ]:
mood.id.nunique()

And there are 19 variables measured, under which the mood which we want to predict

In [ ]:
print(mood.variable.nunique())
print(mood.variable.unique())

In [ ]:
mood.isnull().sum().sort_values(ascending = False)

In [ ]:
mood[(mood.variable.str.contains("circumplex")) & (~mood.value.isnull())]

In [ ]:
mood.head()

In [ ]:
mood[mood.variable=='activity']

We can see that the different variables were started to be logged on different days... We have to deal with that somehow or throw away some lines.

In [ ]:
mood.groupby(['id', 'variable'])['new_date'].min()

In [ ]:
mood[(mood.variable=='mood') & (mood.id=='AS14.01')].value.plot()

## Strategy

We will try for different lengths of period an aggregation of all variables. The aggregation type depends on the variable.

- mood: average of the previous period
- arousal: average of the previous period
- valence: average of the previous period
- activity: average of the previous period
- screen: total screen activity so the sum
- call: number of calls made so the sum
- sms: number of sms so the sum
- for all apps: total duration so the sum


For variables related to duration it seems logical to take the total time for the previous day (or previous days depending on the window). However, it might also be that the screen time before bed impacts the mood the next day.

We define a method to preprocess the data, this includes:
- pivoting the variables so that we get a feature per variable
- aggregating per day
- taking moving averages or sum on a defined backwards period
- define the new target variable as the average mood over the day

**We create a dictionary for aggregation and for windowing so that we can play with the windows later**

In [ ]:
agg_columns = list(mood.variable.unique())
agg_columns

To start we take sum for the durations and mean for the ordinal values.

In [ ]:
agg_dict = {}
for x in agg_columns:
    if x in ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity']:
        agg_dict.update({x: np.nanmean})
    else: agg_dict.update({x:np.nansum})
agg_dict

Lag is 5 days for the ordinal values and 1 for the duration. 

In [ ]:
window_dict = {}
for x in agg_columns:
    if x in ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity']:
        window_dict.update({x: 5})
    else: window_dict.update({x:1})
window_dict

In [ ]:
def aggregate_time_data(dataframe, list_of_deltas, list_of_aggregations):    
    # first pivot the variables so that we get one record per datetime and patient (get out time?)
    pivot_df= dataframe.pivot_table(values='value', index=['id', 'new_date', 'new_time'], \
                                    columns='variable')
    #DO NOT FILL NAN so that pandas only counts the nonzero ones in the means
    #pivot_df.fillna(0, inplace=True)
    #aggregate per day depending on the aggregation type
    daily_df = pivot_df.groupby(['id', 'new_date']).agg(list_of_aggregations)
    #Make mood the target variable
    #daily_df.['target'] = daily_df['mood']
    # shift the other variables 1 day to create new target and take only the previous days into account
    daily_df_shift = daily_df.groupby(['id'])[list(list_of_aggregations.keys())].shift(1)
    #add the target variable (unshifted)
    daily_df_shift['target'] = daily_df['mood']
    #window with the deltas and necessary aggregations
    df_rolling = daily_df_shift
    for x in list_of_deltas.keys():
        df_rolling[x] = daily_df_shift[x].rolling(list_of_deltas.get(x)).agg(list_of_aggregations.get(x))   
    return df_rolling.reset_index(level=['id', 'new_date'])  

In [ ]:
prepped_mood = aggregate_time_data(mood, window_dict, agg_dict)

In [ ]:
prepped_mood

## Base Model

For the base model we want to just take the average mood of the past day, so we process the data only with the shift.

In [ ]:
window_dict_base = {}
for x in agg_columns:
    window_dict_base.update({x:0})
window_dict_base

In [ ]:
prepped_mood_base = aggregate_time_data(mood, window_dict_base, agg_dict)

In [ ]:
pivot_df= mood.pivot_table(values='value', index=['id', 'new_date', 'new_time'], \
                                    columns='variable')
#DO NOT FILL NAN so that pandas only counts the nonzero ones in the means
#pivot_df.fillna(0, inplace=True)
#aggregate per day depending on the aggregation type
daily_mood = pivot_df.groupby(['id', 'new_date']).agg(agg_dict)
# shift the other variables 1 day to create new target and take only the previous days into account
daily_mood_shift = daily_mood.groupby(['id'])[list(agg_dict.keys())].shift(1)
#add the target variable (unshifted)
daily_mood_shift['target'] = daily_mood['mood']
daily_mood_shift = daily_mood_shift.reset_index(level=['id', 'new_date'])

In [ ]:
daily_mood_shift_clean = daily_mood_shift[~np.isnan(daily_mood_shift.mood) & ~np.isnan(daily_mood_shift.target)]
daily_mood_shift_clean['x']= daily_mood_shift_clean['mood']
daily_mood_shift_clean['y']= daily_mood_shift_clean['target']
fig, ax = plt.subplots()
ax.scatter('x', # Horizontal axis
           'y', # Vertical axis
        data=daily_mood_shift_clean, )
ax.plot([3,10], [3,10], color="r", label ="equality")
plt.xlabel("yesterdays average mood")
plt.ylabel("todays average mood")
plt.legend()
plt.title("Visualisation of base model")
plt.show()

In [ ]:
results = daily_mood_shift_clean[[ 'x', 'y']] 

In [ ]:
sns.distplot(results.x-results.y)
plt.title("Prediction errors distribution for the base model")
plt.show()

We can wonder what is a good treshold here. I would argue it doesn t matter that we are less than 1 off. We mainly want to predict the range of mood. Setting the treshold of error to 0.6 already gives us 66%. We could argue if it wouldn 't be better to predict the slope (negative or positive) of mood. One down might be worse than one up, especially from a psychologist perspective. This is not something we will cover in this analysis but a good next step. 

In [ ]:
wrong =len(results[np.abs(results.x - results.y)>0.7])
right = len(results[np.abs(results.x - results.y)<0.7])
total = len(results)
print("We have " + str(right/total) + " right answers")
print("wrong: " + str(wrong))
print("right "+ str(right))

There is quite some activity data missing for certain days

drop the rows with target nan

In [ ]:
prepped_mood = prepped_mood[~np.isnan(prepped_mood.target)]

In [ ]:
prepped_mood[np.isnan(prepped_mood.mood)]

In [ ]:
groups = [0, 1, 2, 3, 5, 6, 7]
i = 1
# plot each column
plt.figure()
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:, group])
    plt.title(dataset.columns[group], y=0.5, loc='right')
    i += 1
plt.show()

## New heading

## Exploring the variables

We perform some basic data analysis on the different variables. Let's first look at the statistics of the numeric variables. 

In [ ]:
prepped_mood.describe()

Most columns are numeric:

In [ ]:
prepped_mood.dtypes.value_counts()

In [ ]:
prepped_mood.describe(include=['O'])

### Numeric variables

We make histograms and boxplots of the different numerical columns. 

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import plotly.figure_factory as ff

app = dash.Dash()

available_indicators = prepped_mood.columns


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            ),
            
            dcc.Graph(id='histograms')
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Graph(id='boxplots')
            
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    
    

#    dcc.Slider(
 #       id='year--slider',
  #      min=df['Year'].min(),
   #     max=df['Year'].max(),
    #    value=df['Year'].max(),
     #   step=None,
      #  marks={str(year): str(year) for year in df['Year'].unique()}
   # )
])

@app.callback(
    dash.dependencies.Output('histograms', 'figure'),
    [dash.dependencies.Input('xaxis-column', 'value'),
     dash.dependencies.Input('xaxis-type', 'value'),
     ])
def update_graph(xaxis_column_name,
                 xaxis_type):
    #dff = df[df['Year'] == year_value]

    return {
        'data': [go.Histogram(
            x=prepped_mood[xaxis_column_name]
            #text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }

@app.callback(
    dash.dependencies.Output('boxplots', 'figure'),
    [dash.dependencies.Input('yaxis-column', 'value'),
    dash.dependencies.Input('yaxis-type', 'value')])


def update_boxplots(yaxis_column_name,
                 yaxis_type):
    #dff = df[df['Year'] == year_value]

     return {
        'data': [go.Box(
            y=prepped_mood[yaxis_column_name]
            #text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
        )],
        'layout': go.Layout(
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }
        
    

if __name__ == '__main__':
    app.run_server()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import plotly.figure_factory as ff

app = dash.Dash()

available_indicators = crimes.columns


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),
    dcc.Graph(id='distributions')

#    dcc.Slider(
 #       id='year--slider',
  #      min=df['Year'].min(),
   #     max=df['Year'].max(),
    #    value=df['Year'].max(),
     #   step=None,
      #  marks={str(year): str(year) for year in df['Year'].unique()}
   # )
])

@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('xaxis-column', 'value'),
     dash.dependencies.Input('yaxis-column', 'value'),
     dash.dependencies.Input('xaxis-type', 'value'),
     dash.dependencies.Input('yaxis-type', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type):
    #dff = df[df['Year'] == year_value]

    return {
        'data': [go.Scatter(
            x=crimes[xaxis_column_name],
            y=crimes[yaxis_column_name],
            #text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'black'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }

@app.callback(
    dash.dependencies.Output('distributions', 'figure'),
    [dash.dependencies.Input('xaxis-column', 'value'),
    dash.dependencies.Input('xaxis-type', 'value')])


def update_distributions(xaxis_column_name,
                 xaxis_type):
    #dff = df[df['Year'] == year_value]

    return ff.create_distplot(
            hist_data = [crimes[xaxis_column_name]],
            group_labels = [crimes.columns],
            rug_text=xaxis_column_name,
            
        )
        
    

if __name__ == '__main__':
    app.run_server()

**Distributions** most variables seem to have exponential or log distribution, not normal. We will have to scale them to perform better in the models

In [ ]:


### Plotting Numeric Features
# Looping through and Plotting Numeric features

for column in prepped_mood.select_dtypes(exclude=['object']).columns:   
    
    # Figure initiation
    fig = plt.figure(figsize=(18,12))
    ### Distribution plot
    sns.distplot(prepped_mood[column].dropna(), ax=plt.subplot(221));
    # X-axis Label
    plt.xlabel(column, fontsize=14);
    # Y-axis Label
    plt.ylabel('Density', fontsize=14);
    ### Boxplot
    sns.boxplot(prepped_mood[column].dropna(), ax=plt.subplot(222));
    # X-axis Label
    plt.xlabel(column, fontsize=14);
    # Y-axis Label
    plt.ylabel("Box", fontsize=14)
    # Adding Super Title (One for a whole figure)
    plt.suptitle('Plots for ' + column, fontsize=18);
 
# Printing Chart
plt.show()

    

### Categorical variables

We make a simple countplot for each variable to see how the records are distributed over the different categories. We see for instance that most places are cities. There are way too many community names to be useful. We see that the data is not sampled evenly over states and over community types (way more data point from cities).

In [ ]:
### Plotting categorical Features
# Looping through and Plotting categorical features


for column in nominal_features:   
    
    # Figure initiation
    fig = plt.figure(figsize=(10,6))
    ### Distribution plot
    sns.countplot(crimes[column].dropna());
    # X-axis Label
    plt.xlabel(column, fontsize=14);
   # Adding Super Title (One for a whole figure)
    plt.suptitle('Count class size for ' + column, fontsize=18);
 
# Printing Chart
plt.show()

## Examing the missing values and dropping columns

We use the missing_no package to visualise the missing values. We see that some columns have too much missing values to be useful.

In [ ]:
msno.matrix(crimes)
plt.show()

In [ ]:
#percentage nulls per column
missing_perc = (crimes.isnull().sum()/crimes.shape[0]).sort_values(ascending = False)

For the columns that have more than half of the values missing we create a new feature indicating whether it is missing or not and drop the original column.

In [ ]:
crimes_helper = crimes[missing_perc[missing_perc.values>0.5].index]

In [ ]:
crimes_helper.columns

In [ ]:
for col in crimes_helper.columns:
    crimes[col+"_missing"] = crimes[col].apply(lambda x: np.isnan(x))

In [ ]:
crimes.columns

In [ ]:
crimes.dropna(thresh = 0.5*len(crimes), axis=1, inplace=True)

In [ ]:
crimes.shape

## Handling missing values

We still have only 1 missing value left in other per cap.

In [ ]:
msno.matrix(crimes)
plt.show()

In [ ]:
#percentage nulls per column
(crimes.isnull().sum()/crimes.shape[0]).sort_values(ascending = False)

only other per cap is missing in one row. But looking at the pct black and such there are no other people than black, whitem asian and hispanic (the sum is alreayd more than hundred).

In [ ]:
crimes.filter(regex="(?i)white|black|asian|other|indian|hisp")[np.isnan(crimes.otherPerCap)]

Let's plot the wages per capita for other. We impute by the median because of the skewedness.

In [ ]:
sns.distplot(crimes.dropna().otherPerCap)
plt.axvline(crimes.dropna().otherPerCap.median(), ls='--', color ='r')
plt.axvline(crimes.dropna().otherPerCap.mean(), ls='--', color ='g')
plt.show()

In [ ]:
crimes["otherPerCap"] = crimes["otherPerCap"].transform(lambda x: x.fillna(x.median()))

In [ ]:
crimes[crimes.otherPerCap.isnull()]

No missing values are left!

In [ ]:
crimes.isnull().sum().sort_values()

## Exploring the correlation between variables

We now explore the correlation between the variables to see which ones we should consider for the models.

In [ ]:
# GETTING Correllation matrix
corr_mat=prepped_mood.corr(method='pearson')
plt.figure(figsize=(20,10))
sns.heatmap(corr_mat,vmax=1,square=True,annot=False,cmap='cubehelix')
plt.title("Correlation between non-class variables")
plt.show()

We want to drop features which are highly correlated:

In [ ]:
# Create correlation matrix
corr_matrix = crimes.drop(columns=['violentPerPop'], axis = 1).corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

#select treshhold

tresh = 0.80

# Find index of feature columns with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > tresh)]

In [ ]:
to_drop

In [ ]:
crimes.drop(crimes[to_drop], axis=1, inplace=True)

In [ ]:
crimes.shape

In [ ]:
# GETTING Correllation matrix
corr_mat=crimes.corr(method='pearson').abs()
plt.figure(figsize=(20,10))
sns.heatmap(corr_mat,vmax=1,square=True,annot=False)
plt.title("Correlation between variables")
plt.show()

## Exploration of correlation with target variable

First, our target variable seems far from normally distributed

In [ ]:
crimes.violentPerPop.plot(kind='HIST', bins = 30)

Log transform looks better

In [ ]:
sns.distplot(crimes['violentPerPop'].apply(lambda x: np.log(x+1))) #plus one to avoid log zero
plt.title("log transformed number of violent crimes")
plt.show()


Make new target variable

In [ ]:
crimes['target'] = crimes['violentPerPop'].apply(lambda x: np.log(x+1))

In [ ]:
crimes

### We look at the categorical variables

State and communityName have way too much 

In [ ]:
### Target variable exploration

facet = sns.FacetGrid(crimes , aspect=4, hue='communityType')
facet.map(sns.kdeplot,'target',shade= True)

plt.legend()
plt.title("Violent crimes per community type")
plt.show()

In [ ]:
sns.factorplot(x='communityType',y='target', hue='communityType', kind="box", data=crimes, size = 7, aspect = 2,)
#limit y axis for visibility


plt.title("violent crimes per community type")




plt.show()

We see that cities have higher violent crime rate.

## Correlation with target for numerical features

We see that the mood for the past 5 days and valence have the highest absolute correlation.

In [ ]:
prepped_mood.corr().abs()['target'].sort_values(ascending = False)

# Regression

## Splitting in train and testset

We use one hot encoding to convert the nominal variables.

In [ ]:
prepped_mood.drop(columns=['new_date'], inplace=True)

In [ ]:
nominal_features = prepped_mood.select_dtypes(include='O')

In [ ]:
nominal_features

We don t need the date anymore now

In [ ]:
prepped_mood = pd.get_dummies(prepped_mood, columns=nominal_features)

In [ ]:
prepped_mood.head()

## Different regression model

### We can consider this problem a regression problem BUT also a classificatino problem ( as the mood is integer and ordinal)

We have to center and scale age and fare. We hence setup an scikit-learn pipeline. We make a list of models to try:
- simple regression
- penalized regression
- Random Forest
- Neural network
- KNN
- XGBoost

We split the dataset in a test and trainset. Our testset size is 50%.

In [ ]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(prepped_mood.drop(axis=1, columns=['target']), prepped_mood['target'], test_size=0.5)


models=dict()
pipelines = dict()
#Logistic Regression
models.update({"LinearRegression": LinearRegression()})
models.update({"Elastic Net": ElasticNet()})   
models.update({"XGBoost": XGBRegressor()})
models.update({"Random Forest": RandomForestRegressor(n_estimators=1000,random_state=0,n_jobs=-1)})
models.update({"Gradient Boost": GradientBoostingRegressor()})
for i in range(2,40):
    models.update({"KNN" + str(i) : KNeighborsRegressor(n_neighbors=i)})
models.update({"Neural Net": MLPRegressor()})

# Compute train and test errors
train_errors = dict()
test_errors = dict()
imputer = Imputer(missing_values="NaN", axis=0,  strategy="mean", verbose=5)
preprocessors = [("imputer", imputer), ('reduce_dim', PCA()), ('scaler',StandardScaler()),('normalizer', Normalizer())]

for label, clf in models.items():
    estimators = preprocessors +  [('clf',clf )]
    pipeline = Pipeline(estimators) #standard scale
    pipelines.update({label:pipeline})
    scores = cross_val_score(
        estimator=pipeline,
        X=X_train,
        y=y_train,
        cv=10,
        scoring='neg_mean_squared_error',
    )
    pipeline.fit(X_test, y_test)
    test_errors.update({label: mean_squared_error(y_test, pipeline.predict(X_test))})
    pipeline.fit(X_train,y_train)
    r2 = r2_score(y_train, pipeline.predict(X_train))
    print(pipeline.feature_importances_)
    print("Neg MSE: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    print("r2: %0.2f [%s]" % (r2, label))
    print("test error: %0.2f" % test_errors.get(label))
    
best_model = min(test_errors.items(), key=operator.itemgetter(1))[0]
print("Best model with lowest test error : %s" % best_model)


In [ ]:
# #############################################################################
# Plot feature importance
clf= pipelines.get("Random Forest").named_steps["clf"]
from yellowbrick.features import FeatureImportances
# Create a new matplotlib figure
fig = plt.figure(figsize=(10,20))
viz = FeatureImportances(clf,)
viz.fit(X_train, y_train)
viz.poof()

In [ ]:
clf=models.get("KNN1")

estimators = [ ('scaler',StandardScaler()),('normalizer', Normalizer()), ('clf',clf )]
pipeline = Pipeline(estimators) #standard scale
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_train)
pd.DataFrame(data={'predictions': y_pred, 'actual': y_train})

R squared: -167521.55 (+/- 42890.81) [LinearRegression]
test error: 205645.63
R squared: -232087.68 (+/- 62426.01) [Elastic Net]
test error: 295655.91
R squared: -148197.50 (+/- 37426.63) [XGBoost]
test error: 76787.41
R squared: -138506.75 (+/- 34178.45) [Random Forest]
test error: 25326.34
R squared: -149016.16 (+/- 34920.26) [Gradient Boost]
test error: 74549.01
R squared: -151117.59 (+/- 35133.19) [KNN]
test error: 164439.03
R squared: -348716.62 (+/- 87084.29) [Neural Net]
test error: 436115.04
Best model with lowest test error : Random Forest

In [ ]:
print(train_errors)
print(test_errors)

In [ ]:


# Instantiate the linear model and visualizer
for label, clf in models.items():
    estimators = [('reduce_dim', PCA()), ('scaler',StandardScaler()),('normalizer', Normalizer()), ('clf',clf )]
    pipeline = Pipeline(estimators) #standard scale    
    visualizer1 = PredictionError(pipeline)
    visualizer1.fit(X_train, y_train)  # Fit the training data to the visualizer
    visualizer1.score(X_test, y_test)
    visualizer = ResidualsPlot(pipeline)
    visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
    visualizer.score(X_test, y_test)  # Evaluate the model on the test data
    g1 = visualizer.poof() # Evaluate the model on the test data
    g = visualizer.poof()             # Draw/show/poof the data

In [ ]:
from yellowbrick.features import FeatureImportances
# Create a new matplotlib figure
fig = plt.figure()
ax = fig.add_subplot()
for label, clf in models.items():
    pipeline = Pipeline([('scaler',StandardScaler()),('normalizer', Normalizer()), ('clf',clf )]) #standard scale
    viz = FeatureImportances(pipeline, ax=ax)
    viz.fit(X_train, y_train)
    viz.poof()

## Performance on the testset

We check now the performance on the testset and see that KNN does not perform very well. Random forest seems the most promising so we will tweak that.

In [ ]:
# #############################################################################
# Compute train and test errors
train_errors = list()
test_errors = list()
for label, clf in models.items():
    .fit(X_train, y_train)
    train_errors.append(enet.score(X_train, y_train))
    test_errors.append(enet.score(X_test, y_test))

i_alpha_optim = np.argmax(test_errors)
alpha_optim = alphas[i_alpha_optim]
print("Optimal regularization parameter : %s" % alpha_optim)

# Estimate the coef_ on full data with optimal regularization parameter
enet.set_params(alpha=alpha_optim)
coef_ = enet.fit(X, y).coef_

# #############################################################################
# Plot results functions

import matplotlib.pyplot as plt
plt.subplot(2, 1, 1)
plt.semilogx(alphas, train_errors, label='Train')
plt.semilogx(alphas, test_errors, label='Test')
plt.vlines(alpha_optim, plt.ylim()[0], np.max(test_errors), color='k',
           linewidth=3, label='Optimum on test')
plt.legend(loc='lower left')
plt.ylim([0, 1.2])
plt.xlabel('Regularization parameter')
plt.ylabel('Performance')

# Show estimated coef_ vs true coef
plt.subplot(2, 1, 2)
plt.plot(coef, label='True coef')
plt.plot(coef_, label='Estimated coef')
plt.legend()
plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.26)
plt.show()

## Different random forest

In [ ]:

rfc = RandomForestRegressor(n_jobs=-1,  oob_score = True) 
pipeline = Pipeline([('scaler',StandardScaler()),('normalizer', Normalizer()), ('clf',rfc )]) #standard scale
# Use a grid over parameters of interest
param_grid = {"clf__n_estimators":[10,100,1000],
    "clf__max_depth": [3, None],
              "clf__max_features": ["sqrt", None, "log2"],
              "clf__min_samples_split": [2, 3, 10],
              "clf__min_samples_leaf": [1, 3, 10],
              # "bootstrap": [True, False],
              #"clf__criterion": ["gini", "entropy"]
             }
#pipeline.get_params().keys()
CV_rfc = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv= 10, scoring = 'mean_squared_error')
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

In [ ]:
# #############################################################################
# Plot feature importance
clf= CV_rfc.best_estimator_.steps[2][1]
from yellowbrick.features import FeatureImportances
# Create a new matplotlib figure
fig = plt.figure(figsize=(10,20))

viz = FeatureImportances(clf,)
viz.fit(X_train, y_train)
viz.poof()

In [ ]:
# Create a list of alphas to cross-validate against
alphas = np.logspace(-12, -0.5, 400)

# Instantiate the linear model and visualizer
model = LassoCV(alphas=alphas)
visualizer = AlphaSelection(model)

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
g = visualizer.poof()             # Draw/show/poof the data